In [ ]:
import math
from pathlib import Path
from typing import Tuple, Callable, List
from IPython.display import display, Math, Image

import cv2
import numba
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp

plt.rcParams['text.usetex'] = True

## Exercise 1:

In [ ]:
display(Image(filename="./images/hw3_p1a.png", height=400, width=500))

In [ ]:
display(Image(filename="./images/hw3_p1b.png", height=400, width=500))

In [ ]:
display(Image(filename="./images/hw3_p1c.png", height=400, width=500))

In [ ]:
display(Image(filename="./images/hw3_p1d.png", height=400, width=500))

# Problem 1d)

With $\lambda_i = 1$, we look at the Eigen Decomposition of $A$:

$$A = Q I Q^{-1} = Q Q^{-1} = I $$

Since we maximized $\mathcal{p} (\mathcal{D} \vert \Sigma)$ with respect to $\lambda_i$, we have:

$$ A = \Sigma^{-1} \tilde{\Sigma} = \Sigma^{-1}_{ML} \tilde{\Sigma} = I $$

Thus,

$$ \Sigma_{ML} = \tilde{\Sigma} $$

### Problem 1f:
An alternative to finding this result, at least numerically would be Gradient Descent.

### Problem 1g:
An unbiased estimate is:

In [ ]:
display(Math(r"\hat{\Sigma}_{unbias} = \frac{1}{N-1} \sum_{n=1}^{N} (\vec{x}_n - \hat{\vec{\mu}})(\vec{x}_n - \hat{\vec{\mu}})^T"))

In [ ]:
# Load the training data
def data_directory() -> Path:
    return Path().cwd() / "data"

train_cat = np.matrix(np.loadtxt(str(data_directory() / "train_cat.txt"), delimiter=","))
train_grass = np.matrix(np.loadtxt(str(data_directory() / "train_grass.txt"), delimiter=","))

print(f"Training Cat Shape: {train_cat.shape}")
print(f"Training Grass Shape: {train_grass.shape}")

## Exercise 2: Bayesian Decision Rule

In [ ]:
display(Image(filename="./images/hw3_p2a.png", height=400, width=500))

### Problem 2b:

In [2]:
gN, gM = train_grass.shape
cN, cM = train_cat.shape



# Estimate means
mu0 = train_grass.mean(axis=1)
mu1 = train_cat.mean(axis=1)
# print(train_grass[1, :].sum() / gM)
# print(mu0)
display(Math(rf"\mu_0: {mu0[0:2]}"))
display(Math(rf"\mu_1: {mu1[0:2]}"))

# Estimate variances
sigma0 = np.cov(train_grass, bias=False)
sigma1 = np.cov(train_cat, bias=False)
display(Math(rf"\Sigma_0:"))
print(sigma0[0:2, 0:2])
# DEBUG:
# i = 1; j = 1
# print(1 / (gM - 1) * ((train_grass[i, :] - mu0[i]) * (train_grass[j, :] - mu0[j]).T))
display(Math(rf"\Sigma_1:"))
print(sigma1[0:2, 0:2])

# Estimate priors
pi0 = gM / (cM + gM)
pi1 = cM / (cM + gM)
display(Math(rf"\pi_0: {pi0}"))
display(Math(rf"\pi_1: {pi1}"))

NameError: name 'train_grass' is not defined

### Problem 2c:

In [ ]:
# Calculate sigma inverses for decision rule
sigma_0_inv = np.linalg.inv(sigma0)
sigma_1_inv = np.linalg.inv(sigma1)
sigma_0_det = np.linalg.det(sigma0)
sigma_1_det = np.linalg.det(sigma1)

@numba.jit
def make_decision(x: np.array) -> int:
    '''Returns 0/1 depending on classification'''
    xmmu0 = (x - mu0)
    xmmu1 = (x - mu1)
    c0 = -1/2 * xmmu0.T @ sigma_0_inv @ xmmu0 + math.log(pi0) - 1/2 * math.log(sigma_0_det)
    c1 = -1/2 * xmmu1.T @ sigma_1_inv @ xmmu1 + math.log(pi1) - 1/2 * math.log(sigma_1_det)
    return 1 if c1 > c0 else 0

@numba.jit
def classify(img: np.matrix) -> np.matrix:
    M, N = img.shape
    P = np.zeros(shape=(M-8, N-8)) - 1 # Initialize prediction matrix to -1's
    for i in range(M-8):
        for j in range(N-8):
            block = img[i:i+8, j:j+8].copy()
            x = block.reshape(64, 1)
            P[i, j] = make_decision(x=x)
    return P

Y = plt.imread(str(data_directory() / "cat_grass.jpg")) / 255
P = classify(Y)

fig = plt.figure()
plt.imshow(Y, cmap=plt.cm.gray)
plt.title("Cat in grass (training)")
fig.gca().axes.xaxis.set_visible(False)
fig.gca().axes.yaxis.set_visible(False)
fig = plt.figure()
plt.imshow(P, cmap=plt.cm.gray)
plt.title("Classification")
fig.gca().axes.xaxis.set_visible(False)
fig.gca().axes.yaxis.set_visible(False)

### Problem 2d:

In [ ]:
# Calculate the Mean Absolute Error (MAE)
Y_truth = plt.imread(str(data_directory() / "truth.png")) / 255
Y_truth[(Y_truth > 0)] = 1.0 # Truth data should be class one where cat is
M, N = Y_truth.shape
Y_truth = Y_truth[0:M-8, 0:N-8]
n_pixels = Y_truth.size
MAE = np.abs(P - Y_truth).sum() / n_pixels
print(f"MAE: {MAE}")

fig = plt.figure()
plt.imshow(Y_truth, cmap=plt.cm.gray)
plt.title("Labeled Truth")
fig.gca().axes.xaxis.set_visible(False)
fig.gca().axes.yaxis.set_visible(False)

### Problem 2e:

In [ ]:
img = cv2.imread(str(data_directory() / "test_wild3.jpeg"))
converted = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
converted: np.matrix = np.asarray(converted) / 255

P2 = classify(converted)
print(P2.max())

fig = plt.figure()
plt.imshow(converted, cmap=plt.cm.gray)
plt.title("Cat in grass (from Google)")
fig.gca().axes.xaxis.set_visible(False)
fig.gca().axes.yaxis.set_visible(False)
fig = plt.figure()
plt.imshow(P2, cmap=plt.cm.gray)
plt.title("Classification")
fig.gca().axes.xaxis.set_visible(False)
fig.gca().axes.yaxis.set_visible(False)

The model performs very poorly as observed above. This is mainly due to the small amount of training data. We estimated the model parameters based on one example of cat + grass. Considering the number of different cat + grass images available, this is not likely to perform well on an out of sample data.

## Exercise 3: Receiver Operating Curve (ROC)

### Problem 3a:

In [ ]:
display(Image(filename="./images/hw3_p3a.png", height=400, width=500))

tau = pi0 / pi1
display(Math(rf"\tau: {tau}, \ log(\tau): {math.log(tau)}"))


### Problem 3b & 3c:

In [ ]:
@numba.jit
def log_likelihood_ratio(x: np.array) -> int:
    '''Returns 0/1 depending on classification'''
    xmmu0 = (x - mu0)
    xmmu1 = (x - mu1)
    c0 = -1/2 * xmmu0.T @ sigma_0_inv @ xmmu0 + math.log(pi0) - 1/2 * math.log(sigma_0_det)
    c1 = -1/2 * xmmu1.T @ sigma_1_inv @ xmmu1 + math.log(pi1) - 1/2 * math.log(sigma_1_det)
    return c1 - c0

@numba.jit
def count_positives(img: np.matrix, truth: np.matrix, tau: float) -> Tuple[int, int]:
    M, N = img.shape
    true_positives = 0
    false_positives = 0
    # llr_values = []
    # Loop over the image and determine if this is a true/false positive
    for i in range(M-8):
        for j in range(N-8):
            block = img[i:i+8, j:j+8].copy()
            x = block.reshape(64, 1)
            llr = log_likelihood_ratio(x=x)
            predicted_class = 1 if llr > tau else 0
            # llr_values.append(llr)
            truth_class = truth[i, j]
            if predicted_class > 0:
                if truth_class > 0:
                    true_positives += 1
                else:
                    false_positives += 1
    # print(max(llr_values), min(llr_values))
    return (true_positives, false_positives)

total_positives = (Y_truth > 0).sum()
total_negatives = (Y_truth == 0).sum()
print(f"Total Positives: {total_positives}, Total Negatives: {total_negatives}")

# Loop over different values of tau
count = 100
pds = []
pfs = []
for tau_i in np.linspace(-365, 50, count):
    tp, fp = count_positives(Y, Y_truth, tau = tau_i)
    pds.append(tp / total_positives)
    pfs.append(fp / total_negatives)

In [ ]:
# Calculate the Bayesian decision point on the ROC
bay_tp, bay_fp = count_positives(Y, Y_truth, tau = tau)
bay_pd = bay_tp / total_positives
bay_pf = bay_fp / total_negatives

fig = plt.figure()
plt.plot(pfs, pds, linewidth=3)
plt.plot(bay_pf, bay_pd, "ro", markersize=8)
plt.legend(["ROC", "Bayesian Decision Point"])
plt.title("ROC")
plt.xlabel(r"$p_F(\tau)$")
plt.ylabel(r"$p_D(\tau)$")

### Problem 3d:

In [ ]:
@numba.jit
def count_positives(img: np.matrix, theta_hat: np.matrix, truth: np.matrix, tau: float) -> Tuple[int, int]:
    M, N = img.shape
    true_positives = 0
    false_positives = 0
    ls_values = [] # Least-Square values
    # Loop over the image and determine if this is a true/false positive
    for i in range(M-8):
        for j in range(N-8):
            block = img[i:i+8, j:j+8].copy()
            x = block.reshape(64, 1)
            ls_value = theta_hat.T @ x
            predicted_class = 1 if ls_value > tau else 0
            ls_values.append(ls_value)
            truth_class = truth[i, j]
            if predicted_class > 0:
                if truth_class > 0:
                    true_positives += 1
                else:
                    false_positives += 1
    # print(max(ls_values), min(ls_values))
    return (true_positives, false_positives)

# Cat stacked on Grass
X = np.vstack((train_cat.T, train_grass.T))
print(X.shape)

b = np.vstack((
    np.ones((train_cat.shape[1], 1)),
    np.ones((train_grass.shape[1], 1)) * -1
))
print(b.shape)

# Solve linear regression problem using cvxpy
d = 64  # theta dimension
theta_hat = cp.Variable((d, 1))
objective = cp.Minimize(cp.sum_squares(X @ theta_hat - b))
constraints = []
prob = cp.Problem(objective, constraints)

optimal_objective_value = prob.solve()
display(Math(r"\hat{\theta} \text{ using cvxpy (first 10 values):}"))
# print(optimal_objective_value)
theta_hat = theta_hat.value
print(theta_hat[:10])

p, fp = count_positives(Y, theta_hat, Y_truth, tau = 1)

# Loop over different values of tau
count = 100
pds = []
pfs = []
for tau_i in np.linspace(-2, 0, count):
    tp, fp = count_positives(Y, theta_hat, Y_truth, tau = tau_i)
    pds.append(tp / total_positives)
    pfs.append(fp / total_negatives)

# Plot the least-squares ROC
fig = plt.figure()
plt.plot(pfs, pds, linewidth=3)
plt.title("Least-Sqaures ROC")
plt.xlabel(r"$p_F(\tau)$")
plt.ylabel(r"$p_D(\tau)$")
